In [1]:
import numpy as np
import pandas as pd

In [2]:
import urllib.request

In [3]:
urlstr = "http://abel.lis.illinois.edu/cgi-bin/cohese/search.py?PMIDs="
Function_words = []

In [4]:
def isNum(str):
    if(any(dig.isdigit() for dig in str) == True):
        return True
    else:
        return False

def parse(s):
    tuples = s.split('), ')
    out = []
    for x in tuples:
        #print(x)
        a,b,c,d = x.strip('[]()""\n').split(', ')
        a = a.replace("'", "")
        d = d.replace("'", "")
        #print(a, b, c, d)
        if(isNum(b) == True):
            b = float(b)
        else:
            b = b.replace("'", "")
        out.append([str(a), b, float(c), str(d)])
    return out

In [5]:
def getListofWords(pin):
    pmidurl = urlstr + str(pin)
    with urllib.request.urlopen(pmidurl) as url:
        string_data = url.read().decode("utf-8")
        list_words = parse(string_data)
        print(pmidurl)
        worddf = pd.DataFrame(list_words, columns=['Word', 'CohessionScore', 'Freq', 'Association'])
        # convert stopwords into cohession score = -1
        worddf.loc[worddf['CohessionScore'] == 'stopword', 'CohessionScore'] = -1
        # remove any remaining spaces
        worddf.Association = worddf.Association.str.strip()
        worddf.Word = worddf.Word.str.strip()
        # convert CohessionScore from object to numberic
        worddf.CohessionScore = worddf.CohessionScore.convert_objects(convert_numeric=True)
        # Filter based on CohessionScore and Association
        filtered_worddf = worddf.loc[(worddf['CohessionScore'] < 0.1)]
        filtered_worddf = filtered_worddf.loc[((filtered_worddf['Association'].str.contains("\\\\N")) | \
                                               (filtered_worddf['Association'].str.contains("None")))]
        #Function_words = Function_words.append(filtered_worddf)
        #if(Function_words.shape[0] % 50 == 0):
        #    print(Function_words.shape[0])
        return filtered_worddf

##### Trial Code

In [150]:
worddf = pd.DataFrame(list_words, columns=['Word', 'CohessionScore', 'Freq', 'Association'])

In [151]:
worddf.loc[worddf['CohessionScore'] == 'stopword', 'CohessionScore'] = -1

In [152]:
worddf.Association = worddf.Association.str.strip()
worddf.Word = worddf.Word.str.strip()

In [154]:
worddf.CohessionScore = worddf.CohessionScore.convert_objects(convert_numeric=True)
worddf.dtypes

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


Word               object
CohessionScore    float64
Freq              float64
Association        object
dtype: object

In [156]:
worddf.head()

,Word,CohessionScore,Freq,Association
0,spatial integration,0.245614,184.0,\\\\N
1,characteristic,0.050822,423070.0,\\\\N
2,were,-1.000000,0.0,None
3,assessed,0.060656,286682.0,\\\\N
4,with,-1.000000,0.0,None


In [163]:
filtered_worddf = worddf.loc[(worddf['CohessionScore'] < 0.1)]
filtered_worddf = filtered_worddf.loc[((filtered_worddf['Association'].str.contains("\\\\N")) | (filtered_worddf['Association'].str.contains("None")))]

##### Run above functions

In [7]:
import time

In [6]:
authors_data = pd.DataFrame.from_csv("D:\\Research\\Data\\soloauthors.tsv", sep='\t')
PMID = authors_data.index
PMID = np.array(PMID)

In [1]:
type(authors_data)

NameError: name 'authors_data' is not defined

In [7]:
PMID[1:100]

array([3940060, 3128160, 3659720, 3310230, 3667770, 3312530, 3318440,
       3318590, 3319230, 2961710, 3696080, 3434650, 3447340, 3275690,
       3342810, 3125250, 3279560, 2451290, 3282770, 3284150, 3284620,
       3373570, 3131880, 3260080, 3390250, 3292080, 3394940, 3042320,
       3139060, 3049740, 3171910, 3171920, 3176070, 2902830, 3052820,
       2972850, 3055200, 3142030, 3056010, 3062030, 3062350, 2463550,
       3213850, 3064280, 3067020, 2852480, 3230510, 3068820, 3072260,
       2909040, 2643420, 2536580, 2921400, 2522140, 2924480, 2926630,
       2564660, 2929790, 2648870, 2651430, 2652250, 2469120, 2540520,
       2565720, 2654890, 2720680, 2729610, 2659480, 2660260, 2525600,
       2500880, 2500990, 2545410, 2526170, 2665590, 2755510, 2474600,
       2764010, 2765190, 2671090, 2570370, 2776030, 2673580, 2676100,
       2677200, 2678550, 2552810, 2805070, 2553730, 2817210, 2819230,
       2684340, 2573430, 2585630, 2586370, 2692630, 2560900, 2633310,
       2517980], dty

In [10]:
from joblib import Parallel, delayed
import multiprocessing

##### Parallel Processing

In [11]:
chunk_size = 500

chunks = [PMID[i:i + chunk_size] for i in range(0, PMID[1:10000].shape[0], chunk_size)]

len(chunks)

20

In [ ]:
num_cores = multiprocessing.cpu_count()
print(num_cores)

# create our pool with `num_processes` processes
pool = multiprocessing.Pool(processes=num_cores)

start_time = time.time()
# apply our function to each chunk in the list
result = pool.map(getListofWords, chunks)


#results = Parallel(n_jobs=num_cores)(delayed(getListofWords)(urlstr + str(pin)) for pin in PMID[1:10])
end_time = time.time()

print(end_time - start_time)

8


In [9]:
PMID[1:30000].shape

(29999,)

In [12]:
start_time = time.time()
result = pd.DataFrame(columns = ['Word', 'CohessionScore', 'Freq', 'Association'])
for ind, pin in enumerate( PMID[1:30000] ):
    result = result.append(getListofWords(urlstr + str(pin)))  
    if(ind % 100 == 0):
        print(ind, " completed...")
        print("Time till now: ", time.time() - start_time)
end_time = time.time()

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if sys.path[0] == '':


0  completed...
Time till now:  0.7677397727966309
100  completed...
Time till now:  78.61974787712097
200  completed...
Time till now:  157.54239153862
300  completed...
Time till now:  235.98217034339905
400  completed...
Time till now:  316.9121022224426
500  completed...
Time till now:  397.7442219257355
600  completed...
Time till now:  479.5858623981476
700  completed...
Time till now:  574.1575951576233
800  completed...
Time till now:  654.766480922699
900  completed...
Time till now:  737.5341105461121
1000  completed...
Time till now:  819.4620816707611
1100  completed...
Time till now:  901.0445265769958
1200  completed...
Time till now:  982.5058181285858
1300  completed...
Time till now:  1065.1923019886017
1400  completed...
Time till now:  1148.183394908905
1500  completed...
Time till now:  1239.0090563297272
1600  completed...
Time till now:  1330.7220339775085
1700  completed...
Time till now:  1424.4471774101257
1800  completed...
Time till now:  1518.133810043335
19

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\pandas\core\ops.py:798: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


TypeError: invalid type comparison

In [14]:
#print(end_time - start_time)
result

,Word,CohessionScore,Freq,Association
1,characteristic,0.050822,423070.0,\\\\N
2,were,-1.000000,0.0,None
3,assessed,0.060656,286682.0,\\\\N
4,with,-1.000000,0.0,None
6,for,-1.000000,0.0,None
7,both,-1.000000,0.0,None
9,and,-1.000000,0.0,None
14,were,-1.000000,0.0,None
16,while,-1.000000,0.0,None
20,both,-1.000000,0.0,None
